In [ ]:
import numpy as np
import pandas as pd
import json
import requests
import pickle
import time
from collections import defaultdict

In [ ]:
in_table= pd.read_csv('ctf_draft - players.csv')
in_table.name = [_.strip().rstrip() for _ in in_table.name]
max_id = 9223372036854775808
id_lookup = {}
pick_lookup = {}
out_table = in_table.set_index('name')
for row in out_table.itertuples():
    if np.isnan(row[1]): continue
    id_lookup[int(row[1])] = row[0]
    pick_lookup[row[0]] = row[2]

In [ ]:
pick_lookup = {row[1]:row[2] for row in pd.read_csv('ctf_draft - ratings.csv').itertuples()}

In [ ]:
ids = np.array(list(id_lookup.keys())).astype(np.uint64)

In [ ]:
weapon_url = 'https://qlstats.net/player/{:d}/weaponstats.json?limit=100&game_type=ctf'

In [ ]:
weapon_stats = {}
with open('ctf_stats.pkl','rb') as fp:
    weapon_stats = pickle.load(fp)
    
for qlid in ids:
    if qlid == max_id:
        continue
    if qlid in weapon_stats:
        continue
    req = requests.get(weapon_url.format(qlid))
    weap = req.json()
    weapon_stats[qlid] = weap
    time.sleep(1.1)

In [ ]:
len(weapon_stats.keys())

In [ ]:
for k,v in weapon_stats.items():
    if len(v['averages']) == 0:
        print(k)
weapon_stats2 = {k:v for k,v in weapon_stats.items() if len(v['averages'])!=0}
weapon_stats = weapon_stats2

In [ ]:
with open('ctf_stats.pkl','wb') as fp:
    pickle.dump(weapon_stats,fp)

In [ ]:
weapon_table = []
for pid, weap_stats in weapon_stats.items():
    w_s = defaultdict(int)
    gs = []
    for ws in weap_stats['weapon_stats']:
        gs.append(ws['game_id'])
        w = ws['weapon_cd']
        w_s[w + '_shots'] += ws['fired']
        w_s[w + '_hits'] += ws['hit']
        w_s[w + '_frags'] += ws['frags']
    gs = len(set(gs))
    w_s = {k:v/gs for k,v in w_s.items()}
    for w in set([_.split('_')[0] for _ in w_s.keys()]):
        w_s[w+'_acc'] = w_s[w+'_hits']/(w_s[w+'_shots']+1e-3)
    w_s['pid'] = pid
    w_s['games'] = gs
    weapon_table.append(w_s)
weapon_table = pd.DataFrame(weapon_table).set_index('pid').fillna(0)

In [ ]:
PRED = (( weapon_table['lg_frags'] / 26.74 ) + ( weapon_table['rl_frags'] / 24.21 ) + ( weapon_table['rg_frags'] / 43.04 ) + ( weapon_table['pg_acc'] * 2.81 ) + ( weapon_table['sg_acc'] * 1.85 ) + ( weapon_table['rg_hits'] / 151.91 ) + ( weapon_table['rl_hits'] / 162.64 ) + ( weapon_table['rg_shots'] / 291.07 ) + ( weapon_table['lg_shots'] / 3466.37 ) + ( weapon_table['sg_frags'] / 40.76 ) + ( weapon_table['rg_acc'] / 1.39 ) + ( weapon_table['rl_acc'] * 1.17 ) + ( weapon_table['gl_shots'] / 291.65 ) + ( weapon_table['lg_hits'] / 2306.52 ) + ( weapon_table['rl_shots'] / 933.4 ) + ( weapon_table['sg_hits'] / 2111.11 ) + ( weapon_table['gt_frags'] / 6.6 ) + ( weapon_table['pg_hits'] / 431.71 ) + ( weapon_table['pg_frags'] / 45.56 ) + ( weapon_table['lg_acc'] / 4.89 ) + ( weapon_table['pg_shots'] / 5830.2 ))
PRED = (PRED-PRED.mean())/PRED.std()
PRED2 = (( weapon_table['lg_hits'] / 8.25 ) + ( weapon_table['lg_shots'] / -26.05 ) + ( weapon_table['sg_shots'] / -26.32 ) + ( weapon_table['rg_hits'] / 1.09 ) + ( weapon_table['sg_hits'] / 8.53 ) + ( weapon_table['lg_frags'] * 1.49 ) + ( weapon_table['sg_frags'] * 1.5 ) + ( weapon_table['rl_hits'] / 2.55 ) + ( weapon_table['rg_shots'] / -5.5 ) + ( weapon_table['mg_shots'] / -32.42 ) + ( weapon_table['rg_frags'] / 1.75 ) + ( weapon_table['pg_hits'] / 3.76 ) + ( weapon_table['mg_acc'] * 74.12 ) + ( weapon_table['mg_hits'] / 15.33 ) + ( weapon_table['gl_hits'] / -0.8 ) + ( weapon_table['gl_shots'] / 10.01 ) + ( weapon_table['rl_frags'] / 2.02 ) + ( weapon_table['pg_shots'] / -53.23 ) + ( weapon_table['pg_acc'] * 26.81 ) + ( weapon_table['rl_shots'] / -40.33 ) + ( weapon_table['sg_acc'] * 13.31 ) + ( weapon_table['gt_frags'] * 2.08 ) + ( weapon_table['mg_frags'] / 2.4 ) + ( weapon_table['rg_acc'] * 6.47 ) + ( weapon_table['pg_frags'] / 2.48 ) + ( weapon_table['lg_acc'] / -0.18 ) + ( weapon_table['rl_acc'] * 5.36 ))
PRED2 = (PRED2-PRED2.mean())/PRED2.std()


PRED_F = 50 + 8*(PRED+PRED2)
weapon_table['PRED_1'] = 50 + 16*(PRED)
weapon_table['PRED_2'] = 50 + 16*(PRED2)

weapon_table['PRED_Total'] = PRED_F

In [ ]:
weapon_table.sort_values('PRED_Total',0,False)

In [ ]:
weapon_table.index = [id_lookup.get(_,str(np.random.randint(3200))) for _ in weapon_table.index ]

In [ ]:
out_t2 = pd.concat([out_table,weapon_table],1)

In [ ]:
out_t2['pick'] = np.array(out_t2.index.map(lambda x: pick_lookup.get(x,'')))
out_t2.sort_values('PRED_Total',0,False)

In [ ]:
out_t2.sort_values('PRED_Total',0,False)[['pick','games','PRED_1','PRED_2','PRED_Total']].to_csv('out_ctf.csv')

In [ ]:
out_t2.sort_values('PRED_Total',0,False)[['pick','games','PRED_Total']]

In [ ]:
dft = out_t2.copy()
true_order = []
for row in dft[['pick']].itertuples():
    if 'c' in row[1]:
        to = -int(row[1][1:])
    elif 'X' in row[1] or '' == row[1]:
        to = 0
        pass
    else:
        to = row[1]
    true_order.append(int(to))
dft['true_o'] = true_order
dft = dft.sort_values('true_o')
dft = dft[dft.true_o!=0]
teamc = list(dft.index[:(dft.true_o < 0).sum()])
teamc

In [ ]:
total_teams = len(teamc)
ti = 0
counter = 1
teams = []
for idx,row in enumerate(dft.itertuples()):
    teams.append(ti)
    print(ti,idx,row[-1],row[0])
    ti+=counter
    
    if ti == total_teams:
        ti = total_teams-1
        counter = -1
    if ti == -1:
        ti = 0
        counter = 1
dft['team'] = teams

In [ ]:
dft

In [ ]:
import matplotlib.pyplot as plt
plt.figure(dpi=120)
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
dft_f = dft[~ np.isnan(dft.PRED_Total)]
dft_f = dft_f[dft.true_o > 0]

pick_names = list(dft_f.sort_values('PRED_Total',0,False).index)
exp_pick = [pick_names.index(_) for _ in dft_f.index]
actual_pick = np.arange(len(exp_pick))
plt.scatter(actual_pick,exp_pick,s=10,alpha=0.5)
plt.grid(True)
plt.xlabel('True pick')
plt.ylabel('Est pick')
plt.title('R = {:.2f}'.format(-dft_f[['PRED_Total','true_o']].corr().iloc[1,0]))

n_d = len(exp_pick)
for i in range(n_d):
    y = int(exp_pick[i])
    plt.text(i,y,dft_f.index[i],size=8,va='center',ha='center')
plt.plot([0,n_d],[0,n_d],c='k',ls='--')

In [ ]:
import sklearn.linear_model as linear
clf = linear.LinearRegression()
dft_f2 = dft.dropna(subset=['PRED_Total'])
clf.fit(np.array(dft_f2.true_o[total_teams:])[:,None],dft_f2.PRED_Total[total_teams:])

In [ ]:
pred_rating = clf.coef_[0] * dft.true_o + clf.intercept_
dft['PRED_S'] = np.where(np.isnan(dft.PRED_Total),pred_rating,dft.PRED_Total)

In [ ]:
draft_res = dft[['team','true_o','PRED_S']].sort_values('PRED_S',0,False)

In [ ]:
team_avgs = defaultdict(list)
team_names = defaultdict(list)
team_cap = dict()
for row in draft_res.iterrows():
    team_avgs[row[1].team].append(row[1].PRED_S)
    team_names[row[1].team].append(row[0])
    if(row[1].true_o < 0):
        team_cap[row[1].team] = row[0]

In [ ]:
team_names

In [ ]:
team_ratings = {}
for i in np.arange(total_teams):
    
    vals = sorted([_ for _ in team_avgs[i] if not np.isnan(_)])[::-1]
    wN = len(vals)
    v = np.arange(wN)+1
    wT = [1 if idx < 4 else 0.4-idx*0.05 for idx in range(wN)]#0.7+0.3*np.tanh(-1*(v-5))
    V = sum([w*v for w,v in zip(wT,vals)])

    
    team_c = sorted([(0 if np.isnan(v) else v,k) for k,v in zip (team_names[i], team_avgs[i])])[::-1]
    team_c = ', '.join(['{} ({:.1f})'.format(v,k) for k,v in team_c])
    
    cap = team_cap[i] + '\n' + team_c
    team_ratings[cap] = V/sum(wT)

In [ ]:
list(zip(wT,vals))

In [ ]:
for i,p in enumerate(sorted([(round(v,1),k) for k,v in team_ratings.items()])[::-1]):
    print('TEAM {}'.format(i+1),p[0],p[1])

In [ ]:
len(team_cap)

In [ ]:
len(team_avgs),len(team_cap)

In [ ]:
team_names[6]

In [ ]:
team_names

In [ ]:
team_cap
